In [105]:
%matplotlib inline

In [106]:
import sys, os, requests, re
from pathlib import Path
import pathlib
streetview_path = Path(os.path.abspath('')).parent.absolute()

if (not (streetview_path in sys.path)) :
    sys.path.append(str(streetview_path))

In [107]:
from xml.dom import minidom
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import numpy as np

import streetview

### Overpass

In [59]:
import overpy
import time
import folium
import numpy as np
import geopandas as gpd

api = overpy.Overpass()

# Effiel Tower (France)
latitude = 48.8583701
longitude = 2.2944813

# Hoan Kiem Lake (Vietname)
# lat = 21.01931
# lng = 105.830341

radius = 200

In [69]:
# https://wiki.openstreetmap.org/wiki/Key:highway#Roads
# phong an

highway_allow = ["motorway","trunk","primary","secondary","tertiary","unclassified",
                 "residential"," motorway_link","trunk_link","primary_link",
                 "secondary_link","tertiary_link","pedestrian"]
allow_list = "|".join(highway_allow)

In [70]:
# Create a map centered on the specified latitude and longitude
m = folium.Map(location=[latitude, longitude], zoom_start=14)

# Query the Overpass API for ways within the specified radius
api = overpy.Overpass()
# result = api.query(f"""
#     way(around:{radius},{latitude},{longitude})[highway];
#     out;
# """)
result = api.query(f"""
    way(around:{radius},{latitude},{longitude})[highway~"^({allow_list})$"];
    out;
""")

In [71]:
waypoint = set()

for way in result.get_ways():
    while True:
        try:
            
            polyline = list()
            
            nodes = way.get_nodes(resolve_missing=True)
            for node in way.nodes:
                polyline.append( (node.lat, node.lon) )
                waypoint.add( (node.lat, node.lon) )
            
            folium.PolyLine(polyline).add_to(m)
        except OverpassTooManyRequests:
            time.sleep(1)
            print('Retrying...')
            continue
        except OverpassGatewayTimeout:
            time.sleep(10)
            print('OverpassGatewayTimeout, retrying...')
            continue
        break

In [91]:
waypoint

{(Decimal('48.8524195'), Decimal('2.2849271')),
 (Decimal('48.8524644'), Decimal('2.2849363')),
 (Decimal('48.8530741'), Decimal('2.2857879')),
 (Decimal('48.8539830'), Decimal('2.2870620')),
 (Decimal('48.8546113'), Decimal('2.2879390')),
 (Decimal('48.8547728'), Decimal('2.2881939')),
 (Decimal('48.8550375'), Decimal('2.2885130')),
 (Decimal('48.8550383'), Decimal('2.2961229')),
 (Decimal('48.8550894'), Decimal('2.2960410')),
 (Decimal('48.8551120'), Decimal('2.2960048')),
 (Decimal('48.8552713'), Decimal('2.2957598')),
 (Decimal('48.8553047'), Decimal('2.2887438')),
 (Decimal('48.8553507'), Decimal('2.2956361')),
 (Decimal('48.8553800'), Decimal('2.2966295')),
 (Decimal('48.8554283'), Decimal('2.2965469')),
 (Decimal('48.8557335'), Decimal('2.2890386')),
 (Decimal('48.8558260'), Decimal('2.2959411')),
 (Decimal('48.8559327'), Decimal('2.2947290')),
 (Decimal('48.8559842'), Decimal('2.2946486')),
 (Decimal('48.8560123'), Decimal('2.2973257')),
 (Decimal('48.8560344'), Decimal('2.2945

In [72]:
m

In [ ]:
def expand_lat_lon(lat_lon: tuple, lat_lon_before: tuple) -> list:
    
    """Expand latitude longitude to get more images between two points"""
    
    lat, lon = lat_lon
    lat_before, lon_before = lat_lon_before
    diff_lat = lat - lat_before
    diff_lon = lon - lon_before
    step_size = 3e-5
    steps_lat = abs(diff_lat) / 1e-5
    steps_lon = abs(diff_lon) / 1e-5
    factor_lat = step_size if diff_lat > 0 else -step_size
    factor_lon = step_size if diff_lon > 0 else -step_size
    jumps = round(max([steps_lat, steps_lon]) / min([steps_lat, steps_lon]))
    expanded = []

    for i in range(int(max([steps_lat, steps_lon]))):

        if not (i % jumps):
            # Find the smaller one
            if steps_lat <= steps_lon:
                lat += factor_lat
            else:
                lon += factor_lon

        # Find the bigger one
        if steps_lat >= steps_lon:
            lat += factor_lat
        else:
            lon += factor_lon
        
        expanded.append((lat, lon))
    
    return expanded


def download_images(coords: list, 
                    show_img: bool=True, 
                    img_dir: str='', 
                    video_dir: str='') -> None:
    """Function to download panorama images from coordinates list"""
    if img_dir:
        img_cnt = 0
        check_path(img_dir)
        
    if video_dir:
        check_path(video_dir)
    
    lat_before = None
    lon_before = None
    pan_before = None
    
    for lat, lon in coords:
        
        if lat_before is not None:
            expanded_lat_lon = expand_lat_lon((lat, lon), (lat_before, lon_before))
        
        if 'expanded_lat_lon' in locals():
            for e_lat, e_lon in expanded_lat_lon:
                print(f"Searching for lat={e_lat}, lon={e_lon}")
                panoids = streetview.panoids(lat=e_lat, lon=e_lon)
                try:
                    panoid = panoids[0]['panoid']
                except IndexError:
                    continue
                panorama = streetview.download_panorama_v3(panoid, zoom=3, disp=False)
                # Don't save the same image twice
                if pan_before is not None:
                    if np.array_equal(panorama, pan_before):
                        continue
                        
                pan_before = panorama
                
                if show_img:
                    plt.figure(figsize=(15, 15))
                    plt.imshow(panorama)
                    plt.axis('off')
                    plt.show()

                # Save img
                if img_dir:
                    # file name format: img_number_latitude_longitude.png
                    plt.imsave(f"{img_dir}/img_{img_cnt}_{lat}_{lon}.jpg", panorama)
                    img_cnt += 1
        
        # Skip first frame
        lat_before = lat
        lon_before = lon
        
    if 'out' in locals():
        out.release()

In [ ]:
coords = get_lat_long(gpx_path)

In [ ]:
download_images(coords, show_img=True, img_dir='./images/', video_dir='./video/')

In [ ]:
panoids = streetview.panoids(lat=40.75388056, lon=-73.99697222)
panoid = panoids[0]['panoid']
panorama = streetview.download_panorama_v3(panoid, zoom=2, disp=False)
plt.imshow(panorama)

In [ ]:
panoid = "ChAzMTM1YWI3NzhlYTEwYmI3"
panorama = streetview.download_panorama_v3(panoid, zoom=2, disp=False)
panorama

### Refacor sv-dlp

In [25]:
from PIL import Image

In [27]:
metadata = None

In [42]:
class MetadataStructure:
    dict_instance = list()
    
    def __init__(self, service=None, pano_id=None, lat=None, lng=None, date=None, size=None, max_zoom=None, misc={}, timeline={}, linked_panos={}):
        self.pano_id = pano_id
        self.lat = lat
        self.lng = lng
        self.date = date
        self.size = size
        self.max_zoom = max_zoom
        self.timeline = timeline
        self.dict_instance.append(self)
        
    def __repr__(self):
        return (
                f"{self.__class__.__name__}("
                f"pano_id={self.pano_id}, "
                f"lat={self.lat}, "
                f"lng={self.lng}, "
                f"date={self.date}, "
                f"size={self.size}, "
                f"max_zoom={self.max_zoom}, "
                f"timeline={self.timeline}, "
        )
        
    @classmethod
    def dict(cls):
        for instance in cls.dict_instance:
            return instance.__dict__

In [44]:
def get_metadata(pano_id=None, lat=None, lng=None, get_linked_panos=False) -> MetadataStructure:
        """
        Calls allocated service's `get_metadata()` function to obtain metadata
        with given input, and store it to class and variable.
        
        Metadata is returned in a `MetadataStructure` object, providing
        the developer a more structured and organized way to handle 
        metadata information with various attributes.
        Additionally, the `.dict()` method returns the attributes of 
        each instance of the MetadataStructure class in the form of a 
        dictionary, allowing for easy access and manipulation of the 
        metadata information.
        
        sv_dlp's metadata structure is designed with compatibility in mind, 
        allowing developers to tinker with it no matter the service picked. 
        An example of the returned metadata is the one below:
        ```python
        metadata = MetadataStructure(
            service=service, 
            pano_id=pano_id, 
            lat=lat, 
            lng=lng, 
            date=datetime.datetime(), 
            size=image_size, 
            max_zoom=max_zoom, 
            timeline=[{'pano_id': 'pano_id', 'date': datetime.datetime()}], 
            linked_panos={{'pano_id': pano_id, 'lat': lat, 'lng': lng, 'date': datetime.datetime()}}, 
            misc={}
        )
        ```
        
        Additionally, the developer has the option to access the metadata 
        in dictionary form by calling the `.dict()` method. An example is:
        ```python
        metadata = {
            "service": service,
            "pano_id": pano_id,
            "lat": lat,
            "lng": lng,
            "date": datetime.datetime(),
            "size": image_size,
            "max_zoom": max_zoom,
            "misc": { # Only use with exclusive service features
                "is_trekker": len(json[1][0][5][0][3][0][0][2]) > 3,
                "gen": gen,
            },
            "timeline": {
                [{'pano_id'}: pano_id, "date": date}],
                [{'pano_id'}: pano_id, "date": date}],
                [{'pano_id'}: pano_id, "date": date}],
                # and so on...
            }
            "linked_panos": {
                [{'pano_id'}: pano_id, "date": date, "lat": lat, "lng" lng}],
                [{'pano_id'}: pano_id, "date": date, "lat": lat, "lng" lng}],
                [{'pano_id'}: pano_id, "date": date, "lat": lat, "lng" lng}],
                # and so on... 
                # only added if get_linked_panos is true
            },
        }
        ```
        
        Parameters
        ----------
        str:    pano_id
            Panorama ID - Might not work with some services
        float:  lat
            Latitude
        float:  lng
            Longitude
        bool:   get_linked_panos
            Sets if linked panos should be returned or not

        Returns
        ----------
        MetadataStructure:  metadata
            Metadata of given input
        MetadataStructure:  self.metadata
            Stores metadata in class
        func:               .dict()
            Function inside object that translates `MetadataStructure` object to dictionary
        """
        md = get_metadata(pano_id=pano_id, lat=lat, lng=lng)
        self.metadata = md
        return md

In [84]:
def _get_pano_from_coords(lat, lng, radius=500) -> dict:
    """
    Returns closest Google panorama ID to given parsed coordinates.
    """
    try:
        url = streetview.urls._build_metadata_url(lat=lat, lng=lng, mode="SingleImageSearch", radius=radius)
        json = requests.get(url).text
        if "Search returned no images." in json:
            print("[google]: Finding nearest panorama via satellite zoom...")
            url = urls._build_metadata_url(lat=lat, lng=lng, mode="SatelliteZoom")
            json = requests.get(url).text
            data = j.loads(json[4:])
            pano = data[1][1][0][0][0][1]
        else:
            data = re.findall(r'\[[0-9],"(.+?)"].+?,\[\[null,null,(.+?),(.+?)\]', json)
            pano = data[0][0]
    except TypeError:
        raise sv_dlp.services.NoPanoIDAvailable
    # pans = re.findall(r"\[[0-9],"(.+?)"].+?,\[\[null,null,(.+?),(.+?)\]", json)
    return pano

In [85]:
_get_pano_from_coords(lat, lng)

_xdc_._b42go0


'PX1BkjvmzSwoov98jBN5HQ'

In [75]:
def _build_tile_arr(panoid = None, zoom=3, alternate=False):

    coord = list(itertools.product(range(imgx), range(13)))

    if alternate:
        image_url = 'https://lh3.ggpht.com/p/{}=x{}-y{}-z{}'
        tiles = [(x, y, "%s_%dx%d.jpg" % (panoid, x, y), image_url.format(panoid, x, y, zoom)) for x, y in coord]
    else:
        image_url = 'https://streetviewpixels-pa.googleapis.com/v1/tile?cb_client=maps_sv.tactile&panoid={}&zoom={}&x={}&y={}'
        tiles = [(x, y, "%s_%dx%d.jpg" % (panoid, x, y), image_url.format(panoid, zoom, x, y)) for x, y in coord]
    
    return tiles

In [51]:
def download_panorama(pano_id=None, lat=None, lng=None, zoom=3) -> Image:
        """
        Obtains Tile URLs List from a given Panorama ID/Coordinate with a
        specified zoom, downloads each row in a multithreaded way and stitches them.
        
        If self.metadata is not allocated (or does not match with given input),
        `get_metadata` is automatically called.
        
        Parameters
        ----------
        str:    pano_id
            Panorama ID - Might not work with some services
        float:  lat
            Latitude
        float:  lng
            Longitude

        Returns
        ----------
        Image:  img
            Stitched Panorama Image in PIL.Image format
        Image:  tile_imgs
            List of Tile Images where each element is stored
            in a PIL.Image format
        Tuple:  tuple
            A tuple is returned if only one
            variable is assigned
        """

        if metadata == None:
            print(f"Obtainin Metadata...")
            if pano_id != None:
                get_metadata(pano_id=pano_id)
            elif lat and lng != None:
                get_metadata(lat=lat, lng=lng)

        if zoom == -1:
            zoom = self.metadata.max_zoom / 2
            zoom = round(zoom + .1)

        print(f"[{self.service_str}]: Building Tile URLs...")
        tile_arr = streetview.urls._build_tile_arr(self.metadata, zoom)
        img, tiles_imgs = download.panorama(tile_arr, self.metadata)
        return img, tiles_imgs


In [54]:
pano = "fzJzOcJLZPq-_QPBJzl5Dg"
lat=6.603079535799973
lng=-73.99819681137278
print(download_panorama(pano))

Obtainin Metadata...


RecursionError: maximum recursion depth exceeded

In [88]:
streetview.metadata._get_panoid_from_coord(latitude, longitude)

AttributeError: module 'streetview' has no attribute 'metadata'

In [37]:
size=[[[[256, 512]], [[512, 1024]], [[1024, 2048]], [[2048, 4096]], [[4096, 8192]], [[8192, 16384]]], [512, 512]]
size[0]

[[[256, 512]],
 [[512, 1024]],
 [[1024, 2048]],
 [[2048, 4096]],
 [[4096, 8192]],
 [[8192, 16384]]]

In [39]:
size[0][3][0][0]

2048

In [40]:
size[1][0]

512

In [41]:
2048 // 512

4

In [102]:
coords = [('lYCBXiB9avkEghp8rW5oWQ', '48.85816212295677', '2.292259828699704'), ('lYCBXiB9avkEghp8rW5oWQ', '48.85816212295677', '2.292259828699704'), ('l2OhUYnXVg8f7C5UkUQkNA', '48.85822687621564', '2.292353127940943'), ('HFF3RXWXIFDV4OOZLX01SA', '48.85802747002063', '2.291747792477793'), ('eXdEqtAxrMpYuxNI24tNZQ', '48.85809096248042', '2.291842700701102'), ('Ftm16xhJWS-3PI_r5rni1g', '48.85815608461488', '2.291940653874237'), ('sDHO_lpzniHaL4wtiKSE7w', '48.85822141986709', '2.292039481572949'), ('9nTL9e82X5RunTcebTgcpg', '48.85828708903684', '2.292136966175553'), ('bbpw-bohPRi_UkfSG-vakQ', '48.85835384554021', '2.292233853890011'), ('jcz48RYOB0uiP78T3gguqg', '48.8584206037313', '2.292330656695505'), ('wdDUAqyBgMi1i0WfTURYyw', '48.85848751870361', '2.292427301451673'), ('wIITz3z19HbKjNKxrwjptA', '48.85855458536836', '2.292524044758148'), ('IEFIAvhy7h2mo8g5H12zrQ', '48.85787585922505', '2.291823612576694'), ('kOLQGAbuwsjeEM5iYbJZVg', '48.85793857467075', '2.291922826937287'), ('De2GPkypnbwX4Zs_IUrcuQ', '48.85800267065402', '2.292022153437689'), ('OsOc9voVYxH7Pp5DoUFoDA', '48.85806612213646', '2.292119115456417'), ('wDjMoIf8FpeO2jEh0qH3MQ', '48.85829171804122', '2.292445835232821'), ('7pgZYbjv7-2FvOXmSkzW3w', '48.85835610476156', '2.292538249320226'), ('HBCNqjSjwyR2E04u091FRA', '48.85842083424689', '2.292632747762904'), ('fXXItxYouTWmx8WENgGjwg', '48.85848594382139', '2.292727448851989'), ('GqPdbBjEnyN3bBeeqA699Q', '48.85809311470431', '2.291734162114328'), ('dAXROEo7VfAkQ-2BInCRfQ', '48.85815860648437', '2.29182897496927'), ('WKvXlX_S7cuNI0F29NyL9g', '48.85822506310517', '2.291925494060548'), ('BVS8Nyybysm8u0PLETpajw', '48.8582582914123', '2.291973753838193'), ('Gh9dLefFk9KPMZVrxF61Jg', '48.85832557134534', '2.292071368395626'), ('8RAUleaFEXtyUp0IVkX_Cw', '48.85839330128138', '2.292169533957144'), ('rOmWH2CFvml3Ij1R0oNl7A', '48.85846127317851', '2.292267120020071'), ('AxTOQJ1mX_zxyIqwSEkljg', '48.85852882559302', '2.292362616747308'), ('RKvn8Oq4f3Z3Al1giCV08g', '48.85803732420326', '2.29167311535068'), ('zwPsiYDrC77k7zN14HkDMQ', '48.85796525219678', '2.291650589110629'), ('-bHT3X3I2vgvo53PaLgO6A', '48.85824484698456', '2.292617659720833'), ('7Ow_pmNUr-NhLNqgVsP3RA', '48.85836203128781', '2.29278286989657'), ('CIkJczbGY-TdDMc8_9kQFg', '48.85792079420321', '2.292623410952928'), ('DzlYMMbvfuY597tWO8oG4g', '48.85831542158429', '2.292716412183751'), ('ErcRFcj-Md1KP1qeKfhfMA', '48.85807905624706', '2.292440714931843'), ('L7yrs702WqHUhG55JxXrTQ', '48.85802649219269', '2.292504057091264'), ('PXu76ESfJd3-N6wklaa-QA', '48.85812476353783', '2.292454426678148'), ('dCKrm24vhxgBirMrt_RbHg', '48.85784031298723', '2.292707977589089'), ('xGD-eZRNG-_uX2fHMJ4GFg', '48.85797249962298', '2.292558720607019'), ('ye8NgFrlUjM6b9Zff_2wMw', '48.85817120770319', '2.292518712271686'), ('5W6yfSbpOFfDYsj_sIZc2A', '48.85816882524027', '2.292266624000511'), ('2qNadrY5p2jr_n_GtcZe0A', '48.85816694269232', '2.292256962665467'), ('-Ruf3YeXqKamJUUNe0A3zQ', '48.85819956709004', '2.292246576683187'), ('Mzpr0dHjParoiPlSMg21kA', '48.85815437270938', '2.292255518803615'), ('xn9d25WhfjRAY9janGnDaQ', '48.85817983772812', '2.292307863773223'), ('KotqdHqn93uaXU1RaYTYUw', '48.85817927334006', '2.292305333775035'), ('l8VkT48s3WPDD6Mf9_BLqA', '48.85817426364947', '2.292263550941798'), ('SvM4nEmLBpLzxAmmQsNf-A', '48.85819923898339', '2.292201145660275'), ('U7XVbucDOiVn91BCxFQTAA', '48.85816071597404', '2.292268795929879'), ('Nke_uFcEhw3rtSt-_wb7BA', '48.85818169959941', '2.292260467924112'), ('txD7N_0i2gTEQHhWh9DIsQ', '48.85812986137158', '2.292213188591407')]

In [104]:
center_lat, center_lon = float(coords[0][1]), float(coords[0][2])

# Create a Folium map centered on the first point
m = folium.Map(location=[center_lat, center_lon], zoom_start=15)

# Add a marker for each coordinate to the map
first_lat, first_lon = float(coords[0][1]), float(coords[0][2])
first_marker = folium.Marker(location=[first_lat, first_lon], color='red')
first_marker.add_to(m)

# Create a list of coordinate pairs for the line
line_coords = [(float(coord[1]), float(coord[2])) for coord in coords[1:]]

# Create a PolyLine object and add it to the map
line = folium.PolyLine(locations=line_coords)
line.add_to(m)

# Display the map
m